# Pre-procesamiento de datos con spaCy

Este tutorial es una selección y adaptación al español del libro [Text Analysis in Python for Social Scientist - Discovery and Exploration](https://www.cambridge.org/core/elements/abs/text-analysis-in-python-for-social-scientists/BFAB0A3604C7E29F6198EA2F7941DFF3) de Dirk Hovy. 

## ¿Qué hay en una palabra?

En esta sección, veremos la terminología para describir algunos de sus elementos básicos (morfología, sintaxis y semántica) y sus equivalentes en el texto (caracteres, palabras, oraciones). 

Para elegir el método correcto para cualquier pregunta de investigación relacionada con el texto que tengamos, tiene sentido pensar qué es el lenguaje y qué no es, cómo está estructurado y cómo "funciona".

Esta sección no reemplaza una introducción a la lingüística. Si estás interesado en leer más sobre esto, hay muchos libros de texto excelentes. Uno de los más entretenidos es Fromkin, Rodman y Hyams (2018).

**El lenguaje a menudo codifica información de manera redundante**, es decir, decimos lo mismo de varias maneras: a través del significado de las palabras, sus posiciones, el contexto y muchas otras señales. 

Las palabras mismas constan de diferentes componentes, que son el foco de diferentes disciplinas lingüísticas: su significado (semántica), su función en una oración (sintaxis) y los prefijos y terminaciones (morfología). 

No todas las palabras contienen toda esta información. Y cuando trabajamos con datos textuales, es posible que no estemos interesados ​​en toda esta información. De hecho, **puede resultar beneficioso eliminar parte de la información que no necesitamos**.

Cuando trabajamos con texto, la unidad que nos interesa depende en gran medida del problema que estamos investigando. Tradicionalmente, se trataba de un informe o un artículo. Sin embargo, cuando trabajamos con redes sociales, también puede referirse al historial de publicaciones completo de un usuario, a un solo mensaje o incluso a una oración individual. 

A lo largo de este tutorial, **nos referiremos a todas estas unidades de texto como documentos**. Debe quedar claro en el contexto qué tamaño tiene un documento. Fundamentalmente, un documento siempre representa una observación en nuestros datos. 

Para referirnos a la colección completa de documentos/observaciones, usamos la palabra **corpus** (corpora plural).

El conjunto de todos los términos únicos en nuestros datos se llama **vocabulario (V)**. Cada elemento de este conjunto se denomina tipo. Cada aparición de un tipo en los datos se denomina **token**. 

Entonces, la oración "una buena oración es una oración que tiene buenas palabras" tiene 10 símbolos pero solo 7 tipos (a saber, "a", "bueno", "oración", "es", "eso", "tiene" y "palabras"). 

Ten en cuenta que los tipos también pueden incluir signos de puntuación y expresiones de varias palabras.

### Descriptores de palabras

#### Tokens y Splitting

Imagina que estamos viendo informes y queremos filtrar oraciones cortas porque no contienen nada de interés. La forma más sencilla de hacerlo es definiendo un límite para el número de palabras. Sin embargo, puede resultar sorprendentemente complicado definir qué es una palabra. 

¿Cuántas palabras hay en “Se fue a Berlín” y en “Se fue a San Luis Obispo”? La definición más general utilizada en muchos idiomas es cualquier cadena de caracteres delimitada por espacios en blanco. 

Sin embargo, ten en cuenta que el chino, por ejemplo, no usa espacios en blanco entre palabras. No todas las palabras están rodeadas de espacios en blanco. Las palabras al principio de una línea no tienen espacios en blanco de antemano. Las palabras al final de una frase u oración pueden tener un símbolo de puntuación (comas, puntos, exclamación o signos de interrogación, etc.) directamente adjunto. 

Comillas y corchetes complican aún más las cosas. Por supuesto, podemos separar estos símbolos introduciendo espacios en blanco adicionales. Este proceso se llama tokenización (porque hacemos que cada palabra y signo de puntuación sea un token separado). 

Un proceso diferente, llamado división de oraciones, separa un documento en oraciones y funciona de manera similar. El problema es decidir cuándo un punto es parte de una palabra (como en títulos como Sr. o abreviaturas como abbr.) O un punto al final de una oración. 

**Tanto la tokenización como la división de oraciones son tareas de predicción**, para las cuales existen modelos confiables de aprendizaje automático. 

No discutiremos el funcionamiento interno de estas herramientas en detalle aquí, sino que nos basaremos en las implementaciones de Python disponibles en el paquete spaCy.

Necesitamos cargar la biblioteca y crear una instancia para el idioma con el que trabajamos (aquí, español), y luego podemos usarlo en cualquier cadena de entrada.

In [1]:
!python -m spacy download es_core_news_sm

In [4]:
import spacy

nlp = spacy.load("es_core_news_sm")
documents = "Estuve 2 veces en Nueva York en 2011, pero no tenía la constitución para ello. No me atrajo. Prefiero Los Angeles."

# Aplicar la división y tokenización de oraciones a un conjunto de documentos.
tokens = [
    [token.text for token in sentence] for sentence in nlp(documents).sents
]
tokens

[['Estuve',
  '2',
  'veces',
  'en',
  'Nueva',
  'York',
  'en',
  '2011',
  ',',
  'pero',
  'no',
  'tenía',
  'la',
  'constitución',
  'para',
  'ello',
  '.'],
 ['No', 'me', 'atrajo', '.'],
 ['Prefiero', 'Los', 'Angeles', '.']]

#### Lematización

Supongamos que tenemos un gran corpus de artículos de la sección comercial de varios periódicos. Nos interesa saber con qué frecuencia y qué empresas se adquieren entre sí. 

Tenemos una lista de palabras, pero las palabras vienen en diferentes formas, según el tiempo y el aspecto, por lo que es posible que tengamos que buscar "adquirir", "adquirió", "adquiere" y "adquirirá" (ya que estamos tratando con artículos de periódicos, pueden estar refiriéndose a eventos recientes o planes futuros, y pueden citar a personas). 

Podríamos intentar formalizar este patrón buscando solo los finales –ir,–rió, –ere y –irá, pero eso solo funciona para verbos regulares que terminan en –ir. Si nos interesan los verbos irregulares, es posible que veamos formas como "ir", "fue", "yendo", o "irá". 

Y no se detiene ahí: las empresas que buscamos pueden adquirir una sola "subsidiaria" o varias "subsidiarias", una "empresa" o varias "empresas". Necesitamos una forma más basada en principios para lidiar con esta variación.

Cuando buscamos una palabra en un diccionario, generalmente buscamos la **forma base** (en el ejemplo anterior, ese sería el infinitivo "ir". Esta forma de base de diccionario se llama **lema**. 

Todas las otras formas no cambian el significado central de este lema, pero agregan más información (como aspectos temporales y de otro tipo. Muchas de estas inflexiones son requeridas por la sintaxis, es decir, el contexto y el orden de las palabras que hacen que una oración sea gramatical. 

Cuando trabajamos con texto y estamos más interesados ​​en el significado que en la morfología o la sintaxis, puede ser útil reemplazar cada palabra con su lema. Este paso reduce la cantidad de variación en los datos y facilita la recopilación de estadísticas significativas. 

En lugar de obtener un solo recuento para cada uno de "ir", "fue", "irá" y "yendo", simplemente registraríamos haber visto la forma "ir" cinco veces en nuestros datos. Esta reducción pierde parte de la información temporal y sintáctica, pero esa información puede ser irrelevante para nuestros propósitos. 

Muchas palabras se pueden lematizar deshaciendo ciertos patrones, según el tipo de palabra. Para las excepciones (por ejemplo, "para ir"), tenemos que tener una tabla de búsqueda.

Afortunadamente, la lematización ya está incorporada en `spaCy`, por lo que podemos hacer uso de ella. Aplicando lematización a nuestras oraciones de ejemplo de arriba, obtenemos:

In [5]:
lemmas = [
    [token.lemma_ for token in sentence] for sentence in nlp(documents).sents
]
lemmas

[['estar',
  '2',
  'vez',
  'en',
  'Nueva',
  'York',
  'en',
  '2011',
  ',',
  'pero',
  'no',
  'tener',
  'el',
  'constitución',
  'para',
  'él',
  '.'],
 ['no', 'yo', 'atraer', '.'],
 ['preferir', 'el', 'Angeles', '.']]

## Partes del discurso


En un nivel muy alto, las palabras denotan cosas, acciones y cualidades en el mundo. Estas categorías corresponden a partes del habla, por ejemplo, sustantivos, verbos y adjetivos (la mayoría de los idiomas tienen más categorías que estas tres). 

Se denominan conjuntamente **palabras de contenido o palabras de clase abierta** porque podemos agregar nuevas palabras a cada una de estas categorías (por ejemplo, nuevos sustantivos, como "tweet", o verbos, como "twerking"). 

Hay otras clases de palabras: determinantes, preposiciones, etc. No "significan" nada (es decir, no se refieren a un concepto), pero ayudan a estructurar una oración y hacerla gramatical. Por lo tanto, se denominan conjuntamente **palabras funcionales** o **palabras de clase cerrada** (es muy poco probable que a alguien se le ocurra una nueva preposición en el corto plazo). 

En parte porque las palabras funcionales son tan cortas y omnipresentes, a menudo se pasan por alto. Si bien tenemos una idea aproximada de cuántas veces hemos visto el sustantivo "clase" en las últimas oraciones, es casi imposible darse cuenta conscientemente con qué frecuencia hemos visto, digamos, "en".

Los idiomas difieren en la forma en que estructuran las oraciones. En consecuencia, hubo poco acuerdo sobre el número exacto de estas categorías gramaticales, más allá de las tres grandes palabras de contenido (e incluso esas no siempre son seguras). 

La necesidad de que las herramientas de NLP funcionen en todos los idiomas generó recientemente esfuerzos para crear un pequeño conjunto de categorías que se apliquen a una amplia gama de idiomas (Petrov, Das y McDonald, 2011). 

Se denominan conjunto de etiquetas de parte del discurso universal (consulte https://universaldependencies.org/u/pos/). 

En este tutorial, usaremos este conjunto de 15 partes del discurso (POS, por sus siglas en inglés).

Palabras de clase abierta:
- ADJ: Adjetivos. Modifican sustantivos para especificar sus propiedades. Ejemplos: impresionante, rojo, aburrido
- ADV: Adverbios. Modifican los verbos, pero también sirven como marcadores de preguntas. Ejemplos: tranquilamente, donde, nunca
- INTJ: Interjecciones. Exclamaciones de algún tipo. Ejemplos: ouch, shhh, oi
- NOUN: Sustantivos. Entidades del mundo. Ejemplos: libro, guerra, tiburón
- PROPN: Nombres propios. Nombres de entidades, una subclase de sustantivos. Ejemplos: Rosa, Twitter, CNN
- VERB: Verbos completos. Eventos en el mundo. Ejemplos: códigos, enviados, correctos

Palabras de clase cerrada:
- ADP: Adposiciones. Preposiciones o posposiciones, marcadores de tiempo, lugar, beneficiario, etc. Ejemplos: terminado, antes, (bajar)
- AUX: Verbos auxiliares y modales. Se usa para cambiar de horario o modalidad. Ejemplos: ha (sido), podría (hacer), será (cambiar)
- CCONJ: Conjunciones coordinadoras. Vincular partes de oraciones con igual importancia. Ejemplos: y, o, pero
- DET: Determinantes. Artículos y cuantificadores. Ejemplos: a, ellos, que
- NUM: Números. Exactamente lo que pensarías que es:::
- PART: Partículas. Posesivos y marcadores gramaticales. Ejemplos: ’s
- PRON: Pronombres. Sustituciones de sustantivos. Ejemplos: tú, ella, él, yo
- SCONJ: Conjunciones subordinadas. Vincula partes de oraciones con una parte que sea más importante. Ejemplos: ya que, si, eso

Otros:
- PUNCT: Signos de puntuación. Ejemplos:!,?, -
- SYM: Símbolos. Entidades parecidas a palabras, a menudo caracteres especiales, incluidos emojis. Ejemplos:%, $, :)
- X: Otro. Todo lo que no se ajuste a ninguno de los anteriores. Ejemplos: pffffrt

La determinación automática de las partes del discurso y la sintaxis de una oración se conoce como etiquetado y análisis de POS, dos de las primeras y más exitosas aplicaciones de NLP. Podemos usar nuevamente el etiquetador POS de `spaCy`:

In [8]:
pos = [[token.pos_ for token in sentence] for sentence in nlp(documents).sents]
pos

[['AUX',
  'NUM',
  'NOUN',
  'ADP',
  'PROPN',
  'PROPN',
  'ADP',
  'NOUN',
  'PUNCT',
  'CCONJ',
  'ADV',
  'VERB',
  'DET',
  'PROPN',
  'ADP',
  'PRON',
  'PUNCT'],
 ['ADV', 'PRON', 'VERB', 'PUNCT'],
 ['VERB', 'DET', 'PROPN', 'PUNCT']]

Debido a que el etiquetado POS fue una de las primeras aplicaciones exitosas de PLN, se han realizado muchas investigaciones al respecto. Por ahora, los etiquetadores de POS son más precisos, más consistentes y definitivamente mucho más rápidos que incluso los lingüistas mejor capacitados.

## Stopwords

Si queremos evaluar los temas generales en las revisiones de productos, generalmente no nos importa si una revisión se refiere al "precio" o "un precio", y podemos eliminar el determinante. 

Del mismo modo, si miramos la posición política de los partidos en sus manifiestos, sabemos quién escribió cada manifiesto. Por tanto, no es necesario que conservemos sus nombres cuando se mencionen a sí mismos en el documento. 

En ambos casos, tenemos un conjunto de palabras que aparecen con frecuencia, pero que no aportan mucho a nuestra tarea, por lo que puede ser beneficioso eliminarlas. El conjunto de estas palabras ignorables es **stopwords**.

Como hemos visto anteriormente, muchas palabras de un texto pertenecen al conjunto de palabras funcionales. De hecho, las palabras más frecuentes en cualquier idioma son predominantemente palabras funcionales. 

Si bien todas estas son palabras muy útiles al construir una oración, en realidad no significan mucho por sí mismas, es decir, sin contexto. De hecho, para la mayoría de las aplicaciones (por ejemplo, modelos de temas; consulte Boyd-Graber, Mimno y Newman, 2014), es mejor ignorarlos por completo.

Podemos excluir las stopwords en función de su parte del discurso (ver más arriba) o mediante el uso de una lista. 

El primero es más general, pero corre el riesgo de descartar a algunos candidatos no deseados. (Si excluimos las preposiciones, corremos el  riesgo de perder el sustantivo "redondo" si se etiqueta incorrectamente). 

El segundo es más específico de la tarea (por ejemplo, podemos usar una lista de nombres de partidos políticos), pero corre el riesgo de omitir las palabras que desconocemos al momento de compilar la lista. 

Por lo tanto, **a menudo puede resultar beneficioso utilizar una combinación de ambos**.

En `spaCy`, podemos usar la propiedad `is_stop` de un token, que lo compara con una lista de palabras vacías comunes.

Para nuestro ejemplo en ejecución, si excluimos las palabras vacías comunes y filtramos las palabras que no contienen contenido (todas las partes del discurso excepto NOUN, VERB, PROPN, ADJ y ADV):

In [10]:
# Seleccionar palabras de contenido a partir de POS.
content = [
    [
        token.text
        for token in sentence
        if token.pos_ in {"NOUN", "VERB", "PROPN", "ADJ", "ADV"}
        and not token.is_stop
    ]
    for sentence in nlp(documents).sents
]
content

[['York', '2011', 'constitución'], ['atrajo'], ['Prefiero', 'Angeles']]

## Expresiones Regulares

Supongamos que está trabajando con una gran muestra de registros de empleo de empleados italianos, con información detallada sobre cuándo trabajaron en qué empresa y una descripción de lo que hicieron allí. 

Sin embargo, los nombres de las empresas figuran en el texto, en lugar de en una entrada separada. Desea extraer los nombres para obtener una descripción general aproximada de cuántas empresas diferentes hay y agrupar los registros en consecuencia. 

Sabe que los nombres de las empresas italianas a menudo terminan en varias abreviaturas (similar a "Co.", "Ltd." o "Inc."). Solo hay dos que desea considerar por ahora ("s.p.a." y "s.r.l."), pero vienen en muchas variaciones de ortografía diferentes (por ejemplo, SPA, spa, S.P.A.). 

¿Cómo puedes identificar estos marcadores y extraer los nombres de las empresas justo antes de ellos?

En el mismo proyecto, ha recopilado datos de encuestas de una gran muestra de empleados. Algunos de ellos han dejado su dirección de correo electrónico como parte de la respuesta y le gustaría detectar y extraer las direcciones de correo electrónico. 

Puedes buscar cualquier cosa que venga antes o después del signo @, pero ¿cómo se asegura de que no sea un identificador de Twitter? ¿Y cómo se asegura de que sea una dirección de correo electrónico válida?

La variación de cadenas es una aplicación de las expresiones regulares (o RegEx). Son patrones flexibles que le permiten especificar lo que está buscando en general y cómo puede variar. Para hacerlo, simplemente escriba el patrón que desea hacer coincidir.

En Python, podemos especificar expresiones regulares y luego aplicarlas al texto con `search()` o `match()`. El primero verifica si un patrón está contenido en la entrada, el segundo verifica si el patrón coincide completamente con la entrada:

In [11]:
import re

pattern = re.compile("at")
print(re.search(pattern, "later"))
print(re.match(pattern, "later"))

<re.Match object; span=(1, 3), match='at'>
None


A veces, sabemos que un caracter específico estará en nuestro patrón, pero no cuántas veces. Podemos usar cuantificadores especiales para señalar que el carácter justo antes de ellos ocurre cualquier número de veces, incluyendo cero (*), exactamente una o cero veces (?), o una o más veces (+). 

Ve los ejemplos en la siguiente tabla:

| Cuantificador 	|  Medios 	| Ejemplo 	|           Matches          	|
|:-------------:	|:-------:	|:-------:	|:--------------------------:	|
|       ?       	|  0 o 1  	|  fr?og  	|          fog, frog         	|
|       *       	| 0 o más 	|  cooo*l 	|         cool, coool        	|
|       +       	| 0 o más 	|  hello+ 	| hello, helloo, hellooooooo 	|

En Python, simplemente los incorporamos a los patrones:

In [12]:
pattern1 = re.compile("fr?og")
pattern2 = re.compile("hello+")
pattern3 = re.compile("cooo*l")

Los RE también proporcionan un conjunto completo de caracteres especiales para coincidir con ciertos tipos de caracteres o posiciones:

| Caracter 	|                           Significado                          	| Ejemplo 	|                  Matches                 	|
|:--------:	|:---------------------------------------------------------:	|:-------:	|:----------------------------------------:	|
|     .    	|                     cualquier caracter                    	|   .el   	|               eel, Nel, gel              	|
|    \n    	|         carácter de nueva línea  (salto de línea)         	|   \n+   	|        uno o más  saltos de línea        	|
|    \t    	|                         tabulación                        	|   \t+   	|          una o más  tabulaciones         	|
|    \d    	|                    un solo dígito \[0-9\]                   	|   B\d   	|              B0, B1, ..., B9             	|
|    \D    	|                        un no digito                       	|   \D.t  	|               ' t, But, eat              	|
|    \w    	|              cualquier carácter  alfanumérico             	|  \w\w\w 	|             top, WOO, bee,...            	|
|    \W    	|                 caracter  no alfanumérico                 	|         	|                                          	|
|    \s    	|             un caracter de  espacio en blanco            	|         	|                                          	|
|    \S    	|        un carácter que no  es un espacio en blanco        	|         	|                                          	|
|     \    	| "Escapa" de los caracteres  especiales para que coincidan 	| .+\.com 	|           abc.com,  united.com           	|
|     ˆ    	|            el comienzo de la  cadena de entrada           	|   ˆ...  	| primera palabra de  tres letras en línea 	|
|     $    	|             el final de la  cadena de entrada             	|   ˆ\n$  	|                linea vacía               	|


Sin embargo, tal vez no queramos realmente ningún carácter antiguo en una posición de comodín, solo uno de un pequeño número de caracteres. Para ello, podemos definir clases de caracteres permitidos:

En Python, podemos usar clases de caracteres para definir nuestros patrones. Mira los ejemplos de la siguiente tabla:

|   Clase  	|                     Significado                     	|     Ejemplo     	|                Matches                	|
|:--------:	|:----------------------------------------------:	|:---------------:	|:-------------------------------------:	|
|  \[abc\] 	|      coincidir con cualquiera  de a, b, c      	|   \[bcrms\]at   	|        bat, cat, rat, mat, sat        	|
| \[ˆabc\] 	|     coincidir con cualquiera  PERO a, b, c     	|    te\[ˆ \]+s   	| tens, tests, teens, texts, terrors... 	|
|  \[a-z\] 	| coincidir con cualquier  carácter en minúscula 	| \[a-z\]\[a-z\]t 	|         act, ant, not, ... wit        	|
|  \[A-Z\] 	| coincidir con cualquier  carácter en mayúscula 	|    \[A-Z\]...   	|      Ahab, Brit, In a, ..., York      	|
|  \[0-9\] 	|         coincidir con  cualquier dígito        	|   DIN A\[0-9\]  	|      DIN A0, DIN A1, ..., DIN A9      	|

In [13]:
pattern = re.compile("[bcr]at")

Hasta ahora, solo hemos definido caracteres individuales y sus repeticiones. Sin embargo, las expresiones regulares también nos permiten especificar grupos completos, como se muestra en la tabla:

|  Grupo  	|              Medios             	|   Ejemplo  	|       Matches       	|
|:-------:	|:-------------------------------:	|:----------:	|:-------------------:	|
|  (abc)  	| secuencia de  coincidencias abc 	|   .(ar).   	| hard, cart, fare... 	|
| (ab\|c) 	|        coincidir  ab o c        	| (ab\|C)ate 	|     abate, Cate     	|

Para aplicar RE a cadenas en Python, tenemos varias opciones. Podemos hacer `match()` a cadenas de entrada, que solo tendrán éxito si pueden cubrir toda la cadena. Si solo nos interesa saber si el patrón está contenido como una subcadena, podemos usar `search()` en su lugar.

In [15]:
document = "el bateador ganó el juego"
matches = re.match(pattern, document)
print(matches)

None


In [16]:
searches = re.search(pattern, document)
print(searches)

<re.Match object; span=(3, 6), match='bat'>


Cualquiera de estas operaciones devuelve un objeto que es None si el patrón no se pudo aplicar. 

De lo contrario, nos proporcionará varias propiedades de los resultados: 
- `span()` nos da una tupla de las posiciones de la subcadena que coinciden con el patrón (y que pueden usarse como índices para cortar) 
- `group()` devuelve la subcadena coincidente. 

Para nuestro ejemplo anterior, `searches.span()` devolverá (3, 6), mientras que `searches.group()` devuelve 'bat':

In [18]:
print(searches.span())
print(searches.group())

(3, 6)
bat


Si definimos varios grupos en nuestro patrón, podemos acceder a ellos a través de `groups()`, ya sea de forma conjunta o individual.

In [18]:
word = "preconstitutionalismo"
affixes = re.compile("(...).+(...)")
results = re.search(affixes, word)
print(results)

<re.Match object; span=(0, 21), match='preconstitutionalismo'>


Aquí, `results.groups()` devolverá el prefijo y el sufijo que se encuentran en la entrada, es decir, ('pre', 'smo'): 


In [20]:
print(results.groups())

('pre', 'smo')


Por último, también podemos usar la expresión regular para reemplazar elementos de la cadena de entrada que coinciden con el patrón a través de `sub()`. 

Hemos mencionado anteriormente cómo podemos reemplazar los dígitos de cualquier número con un token especial, por ejemplo 0. El siguiente código hace esto:

In [19]:
# Reemplazo de patrones en una cadena.
numbers = re.compile("[0-9]")
re.sub(
    numbers,
    "0",
    "En los 90, cuando tenía 12 años, ¡un CD costaba solo 15,99 USD!",
)

'En los 00, cuando tenía 00 años, ¡un CD costaba solo 00,00 USD!'

>Las expresiones regulares son extremadamente poderosas, pero necesita saber cuándo detenerse. Una expresión regular bien escrita puede ahorrarle mucho trabajo y lograr bastante en términos de extraer los fragmentos de texto correctos. Sin embargo, las expresiones regulares pueden volverse extremadamente complicadas rápidamente cuando agrega más y más variantes de las que le gustaría tener en cuenta. Es muy fácil escribir una expresión regular que capture demasiado (falsos positivos) o que sea tan específica que capture muy poco (falsos negativos). Por lo tanto, siempre es una buena idea escribir un par de ejemplos positivos y negativos, y asegurarse de que la expresión regular que desarrolle coincida con lo que desea.

Si tienes tiempo, te invito a revisar el contenido de los libros [Text Analysis in Python for Social Scientists – Discovery and Exploration](https://www.cambridge.org/core/elements/abs/text-analysis-in-python-for-social-scientists/BFAB0A3604C7E29F6198EA2F7941DFF3) y [Twitter as Data](https://www.cambridge.org/core/elements/abs/twitter-as-data/27B3DE20C22E12E162BFB173C5EB2592).